#### [ML] ViT(20.10); Vision Transformer 코드 구현 및 설명 with pytorch



https://kimbg.tistory.com/31

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

In [2]:
from torchvision import transforms, datasets
img_size = 224

# Define image size
image_size = (img_size, img_size)  # Replace with your desired image dimensions

# Create data augmentation transforms
data_augmentation = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor (CHW format)
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize pixel values (common practice)
    transforms.Resize(image_size),  # Resize image to specified dimensions
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip with 50% probability
    transforms.RandomRotation(degrees=(-15, 15)),  # Random rotation with range -15 to 15 degrees
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio=(0.75, 1.3333))  # Random resized crop
])

BATCH_SIZE = 256
train_dataset = datasets.CIFAR100(root="./data/",
                                 train=True,
                                 download=True,
                                 transform=data_augmentation)

test_dataset = datasets.CIFAR100(root="./data/",
                                train=False,
                                download=True,
                                transform=data_augmentation)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)

print(train_loader.dataset)

100%|██████████| 169001437/169001437 [00:10<00:00, 15881416.35it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data/
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
           )


In [3]:
for (x, Y_train) in train_loader:
    print(f"X_train: {x.size()} type: {x.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break

X_train: torch.Size([256, 3, 224, 224]) type: torch.FloatTensor
Y_train: torch.Size([256]) type: torch.LongTensor


In [4]:
## input ##
print('x :', x.shape)

patch_size = 16 # 16x16 pixel patch
patches = rearrange(x, 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', 
                    s1=patch_size, s2=patch_size)
print('patches :', patches.shape)

x : torch.Size([256, 3, 224, 224])
patches : torch.Size([256, 196, 768])


In [5]:
patch_size = 16
in_channels = 3
#emb_size = 768 # channel * patch_size * patch_size
emb_size = 128 # channel * patch_size * patch_size

# using a conv layer instead of a linear one -> performance gains
projection = nn.Sequential(
    nn.Conv2d(in_channels, emb_size, 
              kernel_size=patch_size, stride=patch_size), # torch.Size([8, 768, 14, 14])
    Rearrange('b e (h) (w) -> b (h w) e'))

summary(projection, x.shape[1:], device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 14, 14]          98,432
         Rearrange-2             [-1, 196, 128]               0
Total params: 98,432
Trainable params: 98,432
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 0.38
Params size (MB): 0.38
Estimated Total Size (MB): 1.33
----------------------------------------------------------------


In [6]:
img_size = 224
patch_size = 16

# 이미지를 패치사이즈로 나누고 flatten
projected_x = projection(x)
print('Projected X shape :', projected_x.shape)

# cls_token과 pos encoding Parameter 정의
cls_token = nn.Parameter(torch.randn(1,1, emb_size))
positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))
print('Cls Shape :', cls_token.shape, ', Pos Shape :', positions.shape)

# cls_token을 반복하여 배치사이즈의 크기와 맞춰줌
batch_size = BATCH_SIZE
cls_tokens = repeat(cls_token, '() n e -> b n e', b=batch_size)
print('Repeated Cls shape :', cls_tokens.shape)

# cls_token과 projected_x를 concatenate
cat_x = torch.cat([cls_tokens, projected_x], dim=1)

# position encoding을 더해줌
cat_x += positions
print('output : ', cat_x.shape)

Projected X shape : torch.Size([256, 196, 128])
Cls Shape : torch.Size([1, 1, 128]) , Pos Shape : torch.Size([197, 128])
Repeated Cls shape : torch.Size([256, 1, 128])
output :  torch.Size([256, 197, 128])


In [7]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels: int = 3, patch_size: int = patch_size, 
                 emb_size: int = emb_size, img_size: int = img_size):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # using a conv layer instead of a linear one -> performance gains
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e'))
        self.cls_token = nn.Parameter(torch.randn(1,1, emb_size))
        self.positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))
        
    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        x = self.projection(x)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        # prepend the cls token to the input
        x = torch.cat([cls_tokens, x], dim=1)
        # add position embedding
        x += self.positions

        return x
        
PE = PatchEmbedding()
summary(PE, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 14, 14]          98,432
         Rearrange-2             [-1, 196, 128]               0
Total params: 98,432
Trainable params: 98,432
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 0.38
Params size (MB): 0.38
Estimated Total Size (MB): 1.33
----------------------------------------------------------------


In [8]:
for (x, Y_train) in train_loader:
    print(f"X_train: {x.size()} type: {x.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break
    
#emb_size = 768
num_heads = 8


keys = nn.Linear(emb_size, emb_size)
queries = nn.Linear(emb_size, emb_size)
values = nn.Linear(emb_size, emb_size)
print(keys, queries, values)

x = PE(x)
print(x.shape)
print(queries(x).shape) # batch, n, emb_size
queries = rearrange(queries(x), "b n (h d) -> b h n d", h=num_heads) # -> batch, head, n, emb_size/head
keys = rearrange(keys(x), "b n (h d) -> b h n d", h=num_heads)
values  = rearrange(values(x), "b n (h d) -> b h n d", h=num_heads)

print('shape :', queries.shape, keys.shape, values.shape)

X_train: torch.Size([256, 3, 224, 224]) type: torch.FloatTensor
Y_train: torch.Size([256]) type: torch.LongTensor
Linear(in_features=128, out_features=128, bias=True) Linear(in_features=128, out_features=128, bias=True) Linear(in_features=128, out_features=128, bias=True)
torch.Size([256, 197, 128])
torch.Size([256, 197, 128])
shape : torch.Size([256, 8, 197, 16]) torch.Size([256, 8, 197, 16]) torch.Size([256, 8, 197, 16])


In [9]:
# Queries * Keys
energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)
print('energy :', energy.shape)

# Get Attention Score
scaling = emb_size ** (1/2)
att = F.softmax(energy/scaling, dim=-1) 
print('att :', att.shape)

# Attention Score * values
out = torch.einsum('bhal, bhlv -> bhav ', att, values)
print('out :', out.shape)

# Rearrage to emb_size
out = rearrange(out, "b h n d -> b n (h d)")
print('out2 : ', out.shape)

energy : torch.Size([256, 8, 197, 197])
att : torch.Size([256, 8, 197, 197])
out : torch.Size([256, 8, 197, 16])
out2 :  torch.Size([256, 197, 128])


In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size: int = emb_size, num_heads: int = num_heads, dropout: float = 0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        # fuse the queries, keys and values in one matrix
        self.qkv = nn.Linear(emb_size, emb_size * 3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)
        
    def forward(self, x : Tensor, mask: Tensor = None) -> Tensor:
        # split keys, queries and values in num_heads
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        # sum up over the last axis
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_heads, query_len, key_len
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)
            
        scaling = self.emb_size ** (1/2)
        att = F.softmax(energy / scaling, dim=-1)
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out
        
for (x, Y_train) in train_loader:
    print(f"X_train: {x.size()} type: {x.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break       

PE = PatchEmbedding()
x = PE(x)
print(x.shape)
MHA = MultiHeadAttention()
summary(MHA, x.shape[1:], device='cpu')

X_train: torch.Size([256, 3, 224, 224]) type: torch.FloatTensor
Y_train: torch.Size([256]) type: torch.LongTensor
torch.Size([256, 197, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 197, 384]          49,536
           Dropout-2          [-1, 8, 197, 197]               0
            Linear-3             [-1, 197, 128]          16,512
Total params: 66,048
Trainable params: 66,048
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.10
Forward/backward pass size (MB): 3.14
Params size (MB): 0.25
Estimated Total Size (MB): 3.49
----------------------------------------------------------------


In [11]:
class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
        
    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size: int, expansion: int = 4, drop_p: float = 0.):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )

class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size: int = emb_size,
                 drop_p: float = 0.,
                 forward_expansion: int = 4,
                 forward_drop_p: float = 0.,
                 ** kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))

#x = torch.randn(8, 3, 224, 224)
for (x, Y_train) in train_loader:
    print(f"X_train: {x.size()} type: {x.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break       
x = PE(x)
x = MHA(x)
TE = TransformerEncoderBlock()
summary(TE, x.shape[1:], device='cpu')

X_train: torch.Size([256, 3, 224, 224]) type: torch.FloatTensor
Y_train: torch.Size([256]) type: torch.LongTensor
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1             [-1, 197, 128]             256
            Linear-2             [-1, 197, 384]          49,536
           Dropout-3          [-1, 8, 197, 197]               0
            Linear-4             [-1, 197, 128]          16,512
MultiHeadAttention-5             [-1, 197, 128]               0
           Dropout-6             [-1, 197, 128]               0
       ResidualAdd-7             [-1, 197, 128]               0
         LayerNorm-8             [-1, 197, 128]             256
            Linear-9             [-1, 197, 512]          66,048
             GELU-10             [-1, 197, 512]               0
          Dropout-11             [-1, 197, 512]               0
           Linear-12             [-1, 197, 128]      

In [12]:
num_classes = 100

class TransformerEncoder(nn.Sequential):
    def __init__(self, depth: int = 12, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])

class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size: int = 768, n_classes: int = num_classes):
        super().__init__(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size), 
            nn.Linear(emb_size, n_classes))

class ViT(nn.Sequential):
    def __init__(self,     
                in_channels: int = 3,
                patch_size: int = 16,
                emb_size: int = 768,
                img_size: int = 224,
                depth: int = 12,
                n_classes: int = num_classes,
                **kwargs):
        super().__init__(
            PatchEmbedding(in_channels, patch_size, emb_size, img_size),
            TransformerEncoder(depth, emb_size=emb_size, **kwargs),
            ClassificationHead(emb_size, n_classes)
        )
        
summary(ViT(), (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
         Rearrange-2             [-1, 196, 768]               0
    PatchEmbedding-3             [-1, 197, 768]               0
         LayerNorm-4             [-1, 197, 768]           1,536
            Linear-5            [-1, 197, 2304]       1,771,776
           Dropout-6          [-1, 8, 197, 197]               0
            Linear-7             [-1, 197, 768]         590,592
MultiHeadAttention-8             [-1, 197, 768]               0
           Dropout-9             [-1, 197, 768]               0
      ResidualAdd-10             [-1, 197, 768]               0
        LayerNorm-11             [-1, 197, 768]           1,536
           Linear-12            [-1, 197, 3072]       2,362,368
             GELU-13            [-1, 197, 3072]               0
          Dropout-14            [-1, 19

In [13]:
for (x, Y_train) in train_loader:
    print(f"X_train: {x.size()} type: {x.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break    

vit = ViT()
out = vit(x)
print(out.shape)

X_train: torch.Size([256, 3, 224, 224]) type: torch.FloatTensor
Y_train: torch.Size([256]) type: torch.LongTensor
torch.Size([256, 100])


In [14]:
from torchvision import transforms, datasets
img_size = 224

# Define image size
image_size = (img_size, img_size)  # Replace with your desired image dimensions

# Create data augmentation transforms
data_augmentation = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor (CHW format)
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize pixel values (common practice)
    transforms.Resize(image_size),  # Resize image to specified dimensions
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip with 50% probability
    transforms.RandomRotation(degrees=(-15, 15)),  # Random rotation with range -15 to 15 degrees
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio=(0.8, 1.2))  # Random resized crop
])

test_preprocess = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor (CHW format)
    transforms.Resize(image_size),  # Resize image to specified dimensions 
])

BATCH_SIZE = 256
#BATCH_SIZE = 512

train_dataset = datasets.CIFAR100(root="./data/",
                                 train=True,
                                 download=True,
                                 transform=data_augmentation)

test_dataset = datasets.CIFAR100(root="./data/",
                                train=False,
                                download=True,
                                transform=data_augmentation,
                                #transform=test_preprocess,
                                )

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           #num_workers=4,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          #num_workers=4,
                                          shuffle=False)

print(train_loader.dataset)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.8, 1.2), interpolation=bilinear, antialias=True)
           )


In [ ]:
import torch
from torch import nn
from torch.optim import AdamW

from tqdm.auto import tqdm

# Assuming your model is defined in `model`

num_epochs = 15
weight_decay = 0.001
learning_rate = 0.001


DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = ViT().to(DEVICE)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Define loss function (assuming model outputs logits)
criterion = nn.CrossEntropyLoss()  # For logits

def top_k_accuracy_score(y_true, y_pred, k):
  """
  Calculates top-k accuracy score.

  Args:
      y_true: Ground truth labels (one-hot encoded or integer).
      y_pred: Predicted probabilities (2D array).
      k: The value of k for top-k accuracy.

  Returns:
      Top-k accuracy score.
  """
  correct = 0
  for y_t, y_p in zip(y_true, y_pred):
    # Get top k predictions (indices with highest scores)
    top_k_indices = y_p.argsort()[-k:]

    # Check if true label is in top k predictions
    if y_t in top_k_indices:
      correct += 1

  #accuracy = correct / len(y_true)
  return correct

def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in tqdm(enumerate(train_loader)):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            correct = top_k_accuracy_score(label.clone(), output.clone(), k=1) 
            #accuracy = correct / len(label)
            print(
                f"train Epoch: {Epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)}({100. * batch_idx / len(train_loader):.0f}%)] \
                           \tTrain Loss: {loss.item()} \
                           \tTrain Accuracy: {100. * correct / len(label):.2f}%")


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    top_k_correct = 0 
    #print(f"test_loader length : {len(test_loader.dataset)}") #10000
    with torch.no_grad():
        for image, label in tqdm(test_loader):
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            #print(f"output.shape: {output.shape}") #output.shape: torch.Size([512, 100])
            #print(f"label.shape: {label.shape}") #label.shape: torch.Size([512])
            
            test_loss += criterion(output, label).item()
            #prediction = output.max(1, keepdim=True)[1] 
            #print(prediction.shape) # torch.Size([512, 1])
            #correct += prediction.eq(label.view_as(prediction)).sum().item()
            correct += top_k_accuracy_score(label, output, k=1)
            #print(f"correct: {correct}")
            top_k_correct += top_k_accuracy_score(label, output, k=5)
            #print(f"top_k_correct: {top_k_correct}")

    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    test_top_k_accuracy = 100. * top_k_correct / len(test_loader.dataset)
    return test_loss, test_accuracy, test_top_k_accuracy

for Epoch in range(1, num_epochs + 1):
    train(model, train_loader, optimizer, log_interval=20)
    test_loss, test_accuracy, test_top_k_accuracy = evaluate(model, test_loader)
    print(f"\n[EPOCH: {Epoch}]\tTest Loss: {test_loss:.4f}\tTest Accuracy: {test_accuracy} %\tTest top k Accuracy: {test_top_k_accuracy} % \n")

0it [00:00, ?it/s]

train Epoch: 1 [0/50000(0%)]                            	Train Loss: 4.803012371063232                            	Train Accuracy: 0.00%
train Epoch: 1 [5120/50000(10%)]                            	Train Loss: 4.446662425994873                            	Train Accuracy: 3.12%
train Epoch: 1 [10240/50000(20%)]                            	Train Loss: 4.35072660446167                            	Train Accuracy: 2.73%
train Epoch: 1 [15360/50000(31%)]                            	Train Loss: 4.451959609985352                            	Train Accuracy: 2.34%
train Epoch: 1 [20480/50000(41%)]                            	Train Loss: 4.363228797912598                            	Train Accuracy: 3.52%
train Epoch: 1 [25600/50000(51%)]                            	Train Loss: 4.247077465057373                            	Train Accuracy: 4.30%
train Epoch: 1 [30720/50000(61%)]                            	Train Loss: 4.247827053070068                            	Train Accuracy: 4.69%
train Epoch: 

  0%|          | 0/40 [00:00<?, ?it/s]


[EPOCH: 1]	Test Loss: 0.0158	Test Accuracy: 8.49 %	Test top k Accuracy: 27.79 % 



0it [00:00, ?it/s]

train Epoch: 2 [0/50000(0%)]                            	Train Loss: 3.971151113510132                            	Train Accuracy: 9.38%
train Epoch: 2 [5120/50000(10%)]                            	Train Loss: 3.9428086280822754                            	Train Accuracy: 8.98%
train Epoch: 2 [10240/50000(20%)]                            	Train Loss: 3.9338865280151367                            	Train Accuracy: 12.50%
train Epoch: 2 [15360/50000(31%)]                            	Train Loss: 3.9513208866119385                            	Train Accuracy: 12.50%
train Epoch: 2 [20480/50000(41%)]                            	Train Loss: 3.8655827045440674                            	Train Accuracy: 9.77%
train Epoch: 2 [25600/50000(51%)]                            	Train Loss: 3.6936087608337402                            	Train Accuracy: 10.55%
train Epoch: 2 [30720/50000(61%)]                            	Train Loss: 3.7450931072235107                            	Train Accuracy: 9.38%
tra

  0%|          | 0/40 [00:00<?, ?it/s]


[EPOCH: 2]	Test Loss: 0.0147	Test Accuracy: 12.98 %	Test top k Accuracy: 35.63 % 



0it [00:00, ?it/s]

train Epoch: 3 [0/50000(0%)]                            	Train Loss: 3.6462349891662598                            	Train Accuracy: 13.28%
train Epoch: 3 [5120/50000(10%)]                            	Train Loss: 3.5719211101531982                            	Train Accuracy: 14.06%
train Epoch: 3 [10240/50000(20%)]                            	Train Loss: 3.6494131088256836                            	Train Accuracy: 10.94%
train Epoch: 3 [15360/50000(31%)]                            	Train Loss: 3.6879851818084717                            	Train Accuracy: 12.11%
